In [1]:
!pip install transformers==4.9.1 datasets wandb

     |████████████████████████████████| 2.6 MB 5.1 MB/s 
     |████████████████████████████████| 325 kB 41.5 MB/s 
     |████████████████████████████████| 1.7 MB 43.4 MB/s 
     |████████████████████████████████| 3.3 MB 51.1 MB/s 
     |████████████████████████████████| 895 kB 46.2 MB/s 
     |████████████████████████████████| 596 kB 41.1 MB/s 
     |████████████████████████████████| 212 kB 54.8 MB/s 
     |████████████████████████████████| 134 kB 16.2 MB/s 
     |████████████████████████████████| 312 kB 38.8 MB/s 
     |████████████████████████████████| 311 kB 54.0 MB/s 
     |████████████████████████████████| 312 kB 35.0 MB/s 
     |████████████████████████████████| 311 kB 47.1 MB/s 
     |████████████████████████████████| 311 kB 57.1 MB/s 
     |████████████████████████████████| 306 kB 38.2 MB/s 
     |████████████████████████████████| 298 kB 47.6 MB/s 
     |████████████████████████████████| 298 kB 62.0 MB/s 
     |████████████████████████████████| 290 kB 53.9 MB/s 
     |█████████

In [2]:
import tensorflow as tf
import torch, gc

gc.collect()
torch.cuda.empty_cache()

In [3]:
# GPU 메모리 에러 체크
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [4]:
# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla P100-PCIE-16GB
cuda


In [5]:
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        
Username: valodiel@kakao.com
Password: 
Login successful
Your token: ZzxznowjVkJOJgNxLiuvfEwInLCiwPWlyrmyaKaSYgASEdIMDLMsQPiqFadwiutVUwoLcwXNrAGTdwmwUhgwsLEkaAQnrcupjggDoXnEpEOjPCJcxJwzdxFcyXvsfllP 

Your token has been saved to /root/.huggingface/token


In [6]:
!pip install hf-lfs
!git config --global user.email "esgsky1004@gmail.com"
!git config --global user.name "SeungguEum"

     |████████████████████████████████| 4.0 MB 5.1 MB/s 


In [7]:
import transformers
print(transformers.__version__)

4.9.1


In [8]:
# argment setting
task = "sts"
model_checkpoint = "klue/roberta-large"
batch_size = 32

In [9]:
from datasets import load_dataset
dataset = load_dataset('klue', task)

Downloading:   0%|          | 0.00/5.19k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/sts/1.0.0/55ff8f92b7a4b9842be6514ce0b4b5295b46d5e493f8bb5760da4be717018f90. Subsequent calls will reuse this data.


In [10]:
import re

In [11]:
def cleansing(text):
    text = re.sub('[0-9a-zA-Z-=+,#/\?:^$.@*\"※~&%ㆍ·!』\\‘〈〉|\(\)\[\]\<\>`\'…》《]','', text)
    return text

In [12]:
import torch
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/752k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [13]:
tokenizer('8조 STS 화이팅', '부디 모두의 미래가 행복으로 가득하길')

{'input_ids': [0, 28, 2446, 9468, 2238, 21121, 2, 16321, 3702, 2079, 4037, 2116, 4202, 6233, 4983, 2205, 2454, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
print(f"Sentence1: {dataset['train'][0]['sentence1']}\nSentence2: {dataset['train'][0]['sentence2']}")

Sentence1: 숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.
Sentence2: 숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.


In [15]:
dataset = dataset.flatten()
dataset = dataset.rename_column('labels.real-label','label')

In [16]:
max_length = 512

In [17]:
def preprocess_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'],
                     truncation=True, max_length=max_length, padding=True)

In [18]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [19]:
metric_name = "pearsonr"

In [20]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
num_labels = 1
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,
                                                           num_labels=num_labels)

Downloading:   0%|          | 0.00/547 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classif

In [21]:
import os

model_name = model_checkpoint.split("/")[-1]
output_dir = os.path.join("test-klue", "sts")
logging_dir = os.path.join(output_dir, 'logs')
args = TrainingArguments(
    # checkpoint
    output_dir=output_dir,

    # Model Save & Load
    save_strategy = "epoch",
    load_best_model_at_end=True,

    # Dataset
    num_train_epochs=5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    
    # Optimizer
    learning_rate=2e-5, 
    weight_decay=0.01,  

    # Evaluation 
    metric_for_best_model='eval_' + metric_name,
    evaluation_strategy = "epoch",

    # Logging
    logging_dir=logging_dir,
    report_to='wandb',

    # Randomness
    seed=42,
)

In [22]:
# datasets 라이브러리에서 제공하는 Evaluation metric의 리스트를 확인합니다.
from datasets import list_metrics, load_metric
metrics_list = list_metrics()
len(metrics_list)
print(', '.join(metric for metric in metrics_list))

accuracy, bertscore, bleu, bleurt, cer, chrf, code_eval, comet, competition_math, coval, cuad, f1, frugalscore, gleu, glue, google_bleu, indic_glue, mae, mahalanobis, matthews_correlation, mauve, mean_iou, meteor, mse, pearsonr, perplexity, precision, recall, rouge, sacrebleu, sari, seqeval, spearmanr, squad, squad_v2, super_glue, ter, wer, wiki_split, xnli, xtreme_s


In [23]:
# STS의 metric은 pearsonr, F1 score을 사용합니다.
metric_name = "pearsonr"
metric_pearsonr = load_metric(metric_name) # peason r

Downloading:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

In [24]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions[:, 0]
    pr = metric_pearsonr.compute(predictions=predictions,
                                  references=labels)

    # f1 = metric_f1.compute(predictions=predictions,
    #                               references=labels)
    return pr

In [25]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [26]:
gc.collect()
torch.cuda.empty_cache()

In [27]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence1, labels.label, sentence2, guid, source, labels.binary-label.
***** Running training *****
  Num examples = 11668
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1825
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Pearsonr
1,No log,0.375226,0.871454
2,0.378600,0.437296,0.883623
3,0.118500,0.384789,0.894115
4,0.118500,0.379661,0.896622
5,0.067300,0.333572,0.897340


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence1, labels.label, sentence2, guid, source, labels.binary-label.
***** Running Evaluation *****
  Num examples = 519
  Batch size = 32
Saving model checkpoint to test-klue/sts/checkpoint-365
Configuration saved in test-klue/sts/checkpoint-365/config.json
Model weights saved in test-klue/sts/checkpoint-365/pytorch_model.bin
tokenizer config file saved in test-klue/sts/checkpoint-365/tokenizer_config.json
Special tokens file saved in test-klue/sts/checkpoint-365/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence1, labels.label, sentence2, guid, source, labels.binary-label.
***** Running Evaluation *****
  Num examples = 519
  Batch size = 32
Saving model checkpoint to test-klue/sts/checkpoint-730
Configur

TrainOutput(global_step=1825, training_loss=0.1627422505208891, metrics={'train_runtime': 2417.8016, 'train_samples_per_second': 24.129, 'train_steps_per_second': 0.755, 'total_flos': 1.3366748310623568e+16, 'train_loss': 0.1627422505208891, 'epoch': 5.0})

In [29]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence1, labels.label, sentence2, guid, source, labels.binary-label.
***** Running Evaluation *****
  Num examples = 519
  Batch size = 32


{'epoch': 5.0,
 'eval_loss': 0.3335716426372528,
 'eval_pearsonr': 0.8973397876123219,
 'eval_runtime': 5.6522,
 'eval_samples_per_second': 91.823,
 'eval_steps_per_second': 3.008}

In [30]:
! pip install optuna
! pip install ray[tune]

     |████████████████████████████████| 308 kB 5.3 MB/s 
     |████████████████████████████████| 81 kB 9.4 MB/s 
     |████████████████████████████████| 210 kB 56.9 MB/s 
     |████████████████████████████████| 78 kB 7.3 MB/s 
     |████████████████████████████████| 49 kB 5.6 MB/s 
     |████████████████████████████████| 113 kB 41.2 MB/s 
     |████████████████████████████████| 150 kB 49.9 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=60224e76b81ea64b7983713dd14ae37e6a236098a64cf44230035917954c569b
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip
     |████████████████████████████████| 52.7 MB 49.4 MB/s 
     |████████████████████████████████| 175 kB 39.1 MB/s 
     |████████████████████████████████| 4.1 MB 39.1 MB/s 
     |████████████████████████████████| 125 kB 63.9 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.44.

In [31]:
model_name = model_checkpoint.split("/")[-1]
output_dir = os.path.join("test-klue", "sts-hps")
logging_dir = os.path.join(output_dir, 'logs')

args = TrainingArguments(
    # checkpoint
    output_dir=output_dir,

    # Model Save & Load
    save_strategy = "epoch",
    load_best_model_at_end=True,

    # Dataset
    num_train_epochs=3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    
    # Optimizer
    learning_rate=2e-5,
    weight_decay=0.01,

    # Evaluation 
    metric_for_best_model='eval_pearsonr',
    evaluation_strategy = "epoch",

    # Logging
    logging_dir=logging_dir,

    # Randomness
    seed=42,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [32]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

In [35]:
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=encoded_dataset["train"].shard(index=1, num_shards=10),
    eval_dataset=encoded_dataset['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

loading configuration file https://huggingface.co/klue/roberta-large/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/571e05a2160c18c93365862223c4dae92bbd1b41464a4bd5f372ad703dba6097.ae5b7f8d8a28a3ff0b1560b4d08c6c3bd80f627288eee2024e02959dd60380d0
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertTokenizer",
  "transformers_version": "4.9.1",
  "type_vocab_size": 1,
  "use_cache

In [38]:
def my_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 1, 3),
        "seed": trial.suggest_int("seed", 1, 40),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16, 32, 64]),
    }

In [41]:
gc.collect()
torch.cuda.empty_cache()

In [42]:
best_run = trainer.hyperparameter_search(direction="maximize")

[I 2022-03-21 15:00:25,094] A new study created in memory with name: no-name-ecf4ac7b-0305-4aaf-b365-cb923f91991b
Trial:
loading configuration file https://huggingface.co/klue/roberta-large/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/571e05a2160c18c93365862223c4dae92bbd1b41464a4bd5f372ad703dba6097.ae5b7f8d8a28a3ff0b1560b4d08c6c3bd80f627288eee2024e02959dd60380d0
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": 

Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Pearsonr
1,No log,2.375470,0.145234
2,No log,2.320144,0.179724


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence1, labels.label, sentence2, guid, source, labels.binary-label.
***** Running Evaluation *****
  Num examples = 519
  Batch size = 32
Saving model checkpoint to test-klue/sts-hps/run-0/checkpoint-146
Configuration saved in test-klue/sts-hps/run-0/checkpoint-146/config.json
Model weights saved in test-klue/sts-hps/run-0/checkpoint-146/pytorch_model.bin
tokenizer config file saved in test-klue/sts-hps/run-0/checkpoint-146/tokenizer_config.json
Special tokens file saved in test-klue/sts-hps/run-0/checkpoint-146/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence1, labels.label, sentence2, guid, source, labels.binary-label.
***** Running Evaluation *****
  Num examples = 519
  Batch size = 32
Saving model c

eval/loss,█▁
eval/pearsonr,▁█
eval/runtime,█▁
eval/samples_per_second,▁█
eval/steps_per_second,▁█
train/epoch,▁██
train/global_step,▁██
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁


...

eval/loss,█▃▁
eval/pearsonr,▁▆█
eval/runtime,▁▁█
eval/samples_per_second,██▁
eval/steps_per_second,██▁
train/epoch,▁▃▅██
train/global_step,▁▃▅██
train/learning_rate,▁
train/loss,▁
train/total_flos,▁
train/train_loss,▁


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss


[W 2022-03-21 15:08:56,951] Trial 2 failed because of the following error: RuntimeError('CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 15.90 GiB total capacity; 14.39 GiB already allocated; 31.75 MiB free; 14.76 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF')
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/optuna/study/_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "/usr/local/lib/python3.7/dist-packages/transformers/integrations.py", line 140, in _objective
    trainer.train(resume_from_checkpoint=checkpoint, trial=trial)
  File "/usr/local/lib/python3.7/dist-packages/transformers/trainer.py", line 1280, in train
    tr_loss += self.training_step(model, inputs)
  File "/usr/local/lib/python3.7/dist-packages/transformers/trainer.py", line 1773, in training_ste

RuntimeError: ignored